In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(color_codes=True)

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

In [4]:
df=pd.read_csv("Download/2020_Competition_Training (1).csv")

In [79]:
df.drop('person_id_syn',axis=1, inplace=True)

### Missing Value

In [80]:
df.shape

(69572, 825)

In [81]:
df.loc[:,'cons_cmys']=df['cons_cmys'].apply(lambda x: str(x).split('.')[0])

In [82]:
df['cons_cmys'].value_counts()

4      23539
nan    19267
3      19253
6       3913
5       3404
2        139
0         42
*         15
Name: cons_cmys, dtype: int64

In [83]:
a=df.columns.tolist()
pdc_list=[]
for i in a:
    if 'pdc' in i:
        pdc_list.append(i)
pdc_list

['pdc_ast',
 'pdc_cvd',
 'pdc_dep',
 'pdc_dia',
 'pdc_hf',
 'pdc_ht',
 'pdc_lip',
 'pdc_ost']

In [84]:
L={}
for i in pdc_list:
    L[i]=df[i][df[i]<1.1].median()
L

{'pdc_ast': 1.0,
 'pdc_cvd': 1.0,
 'pdc_dep': 1.0,
 'pdc_dia': 1.0,
 'pdc_hf': 1.0,
 'pdc_ht': 1.0,
 'pdc_lip': 1.0,
 'pdc_ost': 1.0}

In [85]:
L={}
for i in pdc_list:
    L[i]=df[i].median()
L

{'pdc_ast': 1.1,
 'pdc_cvd': 1.1,
 'pdc_dep': 1.1,
 'pdc_dia': 1.1,
 'pdc_hf': 1.0,
 'pdc_ht': 1.0,
 'pdc_lip': 1.1,
 'pdc_ost': 1.1}

In [86]:
# check missing value in row dimension
row_missing = df.isnull().sum(axis=1)/df.shape[1]
print (row_missing[row_missing>0.8])

Series([], dtype: float64)


In [87]:
# create a dataframe to store number of missing value in column dimension
missing = pd.DataFrame(df.isnull().sum(axis=0), columns=['missing_num'])
missing.reset_index(level=0, inplace=True)
missing.head(20)

,index,missing_num
0,transportation_issues,0
1,src_platform_cd,0
2,sex_cd,0
3,est_age,0
4,smoker_current_ind,0
5,smoker_former_ind,0
6,lang_spoken_cd,0
7,mabh_seg,0
8,cci_score,0
9,dcsi_score,0


In [88]:
# count missing values for pdc features 
pdc={}
for i in df.columns:
    if "pdc" in i:
        pdc[i]=0
        for index, row in df.iterrows():
            if row[i]==1.1:
                pdc[i]+=1
pdc

{'pdc_ast': 59624,
 'pdc_cvd': 65117,
 'pdc_dep': 51422,
 'pdc_dia': 56120,
 'pdc_hf': 22384,
 'pdc_ht': 22393,
 'pdc_lip': 34956,
 'pdc_ost': 66291}

In [89]:
# Count missing values for zip/cnty/state  ['zip_cd', 'cnty_cd', 'state_cd']
M={}
L=['zip_cd', 'cnty_cd', 'state_cd']
for i in df.columns:
    if i in L:
        M[i]=0
        for index, row in df.iterrows():
            if row[i]=='other':
                M[i]+=1
M

{'zip_cd': 46773, 'cnty_cd': 46773, 'state_cd': 46773}

In [90]:
# update 'missing' table with new pdc missing value 
pdc_feature_index=[]
for index, row in missing.iterrows():
    feature=row['index']
    if feature in pdc.keys():
        pdc_feature_index.append(index)
for i in pdc_feature_index:
    feature=missing.loc[i,'index']
    missing.loc[i,'missing_num']=pdc[feature]

In [91]:
# 80% or more missing value for feature dimension
num_80 = 0.8 * df.shape[0]
missing_80_plus = missing[missing['missing_num'] >= num_80].sort_values(by=['missing_num'], ascending=False)

missing_80_plus

,index,missing_num
192,hedis_ami,69339
236,pdc_ost,66291
230,pdc_cvd,65117
229,pdc_ast,59624
232,pdc_dia,56120


In [92]:
missing[missing['missing_num']>300].sort_values(by=['missing_num'], ascending=False)

,index,missing_num
192,hedis_ami,69339
236,pdc_ost,66291
230,pdc_cvd,65117
229,pdc_ast,59624
232,pdc_dia,56120
193,hedis_cmc_ldc_c_control,54932
194,hedis_cmc_ldc_c_screen,54930
231,pdc_dep,51422
235,pdc_lip,34956
234,pdc_ht,22393


#### PDC Missing value

In [93]:
pdc_list=missing[missing['index'].str.contains('pdc')]['index'].tolist()

In [6]:
# Use 0 to replace 1.1
df['pdc_ast'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_cvd'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_dep'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_dia'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_hf'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_ht'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_lip'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_ost'].apply(lambda x: 1 if x==1.1 else x)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
69567    1.0
69568    1.0
69569    1.0
69570    1.0
69571    1.0
Name: pdc_ost, Length: 69572, dtype: float64

In [96]:
#check
L={}
for i in pdc_list:
    L[i]=df[i].median()
L

{'pdc_ast': 1.0,
 'pdc_cvd': 1.0,
 'pdc_dep': 1.0,
 'pdc_dia': 1.0,
 'pdc_hf': 1.0,
 'pdc_ht': 1.0,
 'pdc_lip': 1.0,
 'pdc_ost': 1.0}

#### hedis missing value

In [97]:
# drop column (missing value>80% & categorical) 
# Since other columns with missing values are all categorical, we will perform one-hot encoding later
df.drop(['hedis_ami'], axis=1, inplace=True) 

#### cons missing value

In [98]:
# for categorical features, we will perform one-hot encoding ['cons_hhcomp','cons_homstat','cons_cmys']
# for numerical features, we will first perform data interpretation 
# pick out the numerical features
cons_list=missing[missing['index'].str.contains('cons_')]['index'].tolist() 
cons_list.remove('cons_hhcomp') ### 咋一次性remove好几个来着？
cons_list.remove('cons_homstat')
cons_list.remove('cons_cmys')
cons_list.remove('submcc_sns_cons_pmpm_ct')
cons_list.remove('submcc_sns_cons_ind')
cons_list

['cons_hcaccprf_h',
 'cons_hcaccprf_p',
 'cons_n2029_y',
 'cons_n2mob',
 'cons_n2pbl',
 'cons_n2pmv',
 'cons_n65p_y',
 'cons_online_buyer',
 'cons_ret_y',
 'cons_retail_buyer',
 'cons_veteran_y']

In [99]:
df[cons_list].describe().transpose()[['mean','50%','std','min','max']]

,mean,50%,std,min,max
cons_hcaccprf_h,0.014037,0.0,0.117646,0.0,1.0
cons_hcaccprf_p,0.907504,1.0,0.289728,0.0,1.0
cons_n2029_y,0.060784,0.0,0.238936,0.0,1.0
cons_n2mob,9.351215,1.0,15.556647,0.0,99.0
cons_n2pbl,16.338470,5.0,24.299629,0.0,99.0
cons_n2pmv,92.247167,95.0,7.972552,0.0,99.0
cons_n65p_y,0.763212,1.0,0.425115,0.0,1.0
cons_online_buyer,0.231366,0.0,0.421710,0.0,1.0
cons_ret_y,0.187263,0.0,0.390126,0.0,1.0
cons_retail_buyer,0.186766,0.0,0.389727,0.0,1.0


In [100]:
# impute NA with median value since some features have extreme values
for f in cons_list:
    df[f].fillna(np.nanmedian(df[f]), inplace=True)

#### credit & cms missing value 

In [101]:
cms_list=missing[missing['index'].str.contains('cms_')]['index'].tolist()
credit_list=missing[missing['index'].str.contains('credit_')]['index'].tolist()

In [102]:
df[cms_list].describe().transpose()[['mean','50%','std','min','max']]

,mean,50%,std,min,max
cms_disabled_ind,0.302076,0.000,0.459161,0.00,1.000
cms_dual_eligible_ind,0.164448,0.000,0.370685,0.00,1.000
cms_hospice_ind,0.000517,0.000,0.022742,0.00,1.000
cms_low_income_ind,0.237567,0.000,0.425595,0.00,1.000
cms_ma_risk_score_nbr,1.121419,0.870,0.904994,0.00,9.437
cms_partd_ra_factor_amt,1.071810,0.890,0.842034,0.00,12.151
cms_risk_adj_payment_rate_a_amt,398.052546,299.785,363.614271,0.00,4984.320
cms_risk_adj_payment_rate_b_amt,506.892163,389.300,465.931442,0.00,6989.850
cms_risk_adjustment_factor_a_amt,1.119722,0.870,0.923938,0.00,9.437
cms_rx_risk_score_nbr,1.049278,0.877,0.824906,0.00,11.706


In [103]:
df[credit_list].describe().transpose()[['mean','50%','std','min','max']]

,mean,50%,std,min,max
credit_bal_1stmtg_30to59dpd,815.877674,726.071464,450.947418,0.000000,5522.954198
credit_bal_1stmtg_60to89dpd,284.567259,244.426559,197.414608,0.000000,3896.254369
credit_bal_1stmtg_collections,411.440412,325.429098,347.308388,0.000000,5182.580292
credit_bal_1stmtg_severederog,48.438003,33.398561,67.397612,0.000000,2550.013846
credit_bal_1stmtgcredit_60dpd,1272.639332,1038.697018,855.546328,0.060000,11974.587292
...,...,...,...,...,...
credit_num_totalallcredit_severederog,0.396132,0.377939,0.163456,0.041538,1.357202
credit_prcnt_agencyfirstmtg,58.801967,58.713733,9.136879,12.512821,88.304536
credit_prcnt_autobank,57.804801,58.015975,3.938653,26.972855,71.607280
credit_prcnt_mtgcredit,61.931256,61.978195,6.457041,30.498993,88.131621


In [5]:
# impute NA with median value
df=df.fillna(df.median())

In [105]:
#missing value check
missing2 = pd.DataFrame(df.isnull().sum(axis=0), columns=['missing_num'])
missing2.reset_index(level=0, inplace=True)
missing2[missing2['missing_num']>0].sort_values(by=['missing_num'], ascending=False)

,index,missing_num
192,hedis_cmc_ldc_c_control,54932
193,hedis_cmc_ldc_c_screen,54930
83,cons_homstat,19280
82,cons_hhcomp,19277
63,cms_ra_factor_type_cd,4224
196,hedis_dia_hba1c_test,1095
199,hedis_dia_ma_nephr,1094
195,hedis_dia_hba1c_ge9,1093
197,hedis_dia_ldc_c_control,1093
198,hedis_dia_ldc_c_screen,1093


### Get Dummy

In [106]:
# drop categorical feature
no_hoc_cat=['rucc_category','hedis_cmc_ldc_c_control','hedis_cmc_ldc_c_screen',
            'hedis_dia_eye','hedis_dia_hba1c_ge9','hedis_dia_hba1c_test','hedis_dia_ldc_c_control',
            'hedis_dia_ldc_c_screen','hedis_dia_ma_nephr','cms_ra_factor_type_cd',
            'mabh_seg','lang_spoken_cd','sex_cd','src_platform_cd',
            'cons_hhcomp','cons_homstat','cons_cmys']
df_1hot=pd.DataFrame()
for s in no_hoc_cat:
    df_temp = pd.get_dummies(df[s], prefix = s)
    df_1hot = pd.concat([df_1hot,df_temp], axis=1)

In [107]:
df=pd.concat([df, df_1hot], axis=1)
df.drop(no_hoc_cat, axis=1, inplace=True) 